# Lab 22: Interpreting Results and Confidence Measures

## Overview

In this lab, we'll explore how Bonsai v3 interprets genetic data to make relationship predictions and assigns confidence levels to these predictions. We'll examine the statistical models, likelihood calculations, and confidence measures that help users understand the reliability of inferred relationships and pedigree structures.

In [ ]:
# 🧬 Google Colab Setup - Run this cell first!
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
from IPython.display import display, HTML, Markdown

def is_colab():
    '''Check if running in Google Colab'''
    try:
        import google.colab
        return True
    except ImportError:
        return False

if is_colab():
    print("🔬 Setting up Google Colab environment...")
    
    # Install dependencies
    print("📦 Installing packages...")
    !pip install -q pysam biopython scikit-allel networkx pygraphviz seaborn plotly
    !apt-get update -qq && apt-get install -qq samtools bcftools tabix graphviz-dev
    
    # Create directories
    !mkdir -p /content/class_data /content/results
    
    # Download essential class data
    print("📥 Downloading class data...")
    S3_BASE = "https://computational-genetic-genealogy.s3.us-east-2.amazonaws.com/class_data/"
    data_files = [
        "pedigree.fam", "pedigree.def", 
        "merged_opensnps_autosomes_ped_sim.seg",
        "merged_opensnps_autosomes_ped_sim-everyone.fam",
        "ped_sim_run2.seg", "ped_sim_run2-everyone.fam"
    ]
    
    for file in data_files:
        !wget -q -O /content/class_data/{file} {S3_BASE}{file}
        print(f"  ✅ {file}")
    
    # Define utility functions
    def setup_environment():
        return "/content/class_data", "/content/results"
    
    def save_results(dataframe, filename, description="results"):
        os.makedirs("/content/results", exist_ok=True)
        full_path = f"/content/results/{filename}"
        dataframe.to_csv(full_path, index=False)
        display(HTML(f'''
        <div style="padding: 10px; background-color: #e3f2fd; border-left: 4px solid #2196f3; margin: 10px 0;">
            <p><strong>💾 Results saved!</strong> To download: 
            <code>from google.colab import files; files.download('{full_path}')</code></p>
        </div>
        '''))
        return full_path
    
    def save_plot(plt, filename, description="plot"):
        os.makedirs("/content/results", exist_ok=True)
        full_path = f"/content/results/{filename}"
        plt.savefig(full_path, dpi=300, bbox_inches='tight')
        plt.show()
        display(HTML(f'''
        <div style="padding: 10px; background-color: #e8f5e8; border-left: 4px solid #4caf50; margin: 10px 0;">
            <p><strong>📊 Plot saved!</strong> To download: 
            <code>from google.colab import files; files.download('{full_path}')</code></p>
        </div>
        '''))
        return full_path
    
    print("✅ Colab setup complete! Ready to explore genetic genealogy.")
    
else:
    print("🏠 Local environment detected")
    def setup_environment():
        return "class_data", "results"
    def save_results(df, filename, description=""):
        os.makedirs("results", exist_ok=True)
        path = f"results/{filename}"
        df.to_csv(path, index=False)
        return path
    def save_plot(plt, filename, description=""):
        os.makedirs("results", exist_ok=True)
        path = f"results/{filename}"
        plt.savefig(path, dpi=300, bbox_inches='tight')
        plt.show()
        return path

# Set up paths and configure visualization
DATA_DIR, RESULTS_DIR = setup_environment()
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_context("notebook")

In [ ]:
# Setup Bonsai module paths
if not is_jupyterlite():
    # In local environment, add the utils directory to system path
    utils_dir = os.getenv('PROJECT_UTILS_DIR', os.path.join(os.path.dirname(DATA_DIR), 'utils'))
    bonsaitree_dir = os.path.join(utils_dir, 'bonsaitree')
    
    # Add to path if it exists and isn't already there
    if os.path.exists(bonsaitree_dir) and bonsaitree_dir not in sys.path:
        sys.path.append(bonsaitree_dir)
        print(f"Added {bonsaitree_dir} to sys.path")
else:
    # In JupyterLite, use a simplified approach
    print("⚠️ Running in JupyterLite: Some Bonsai functionality may be limited.")
    print("This notebook is primarily designed for local execution where the Bonsai codebase is available.")

In [ ]:
# Helper functions for exploring modules
def display_module_classes(module_name):
    """Display classes and their docstrings from a module"""
    try:
        # Import the module
        module = importlib.import_module(module_name)
        
        # Find all classes
        classes = inspect.getmembers(module, inspect.isclass)
        
        # Filter classes defined in this module (not imported)
        classes = [(name, cls) for name, cls in classes if cls.__module__ == module_name]
        
        # Print info for each class
        for name, cls in classes:
            print(f"\n## {name}")
            
            # Get docstring
            doc = inspect.getdoc(cls)
            if doc:
                print(f"Docstring: {doc}")
            else:
                print("No docstring available")
            
            # Get methods
            methods = inspect.getmembers(cls, inspect.isfunction)
            if methods:
                print("\nMethods:")
                for method_name, method in methods:
                    if not method_name.startswith('_'):  # Skip private methods
                        print(f"- {method_name}")
    except ImportError as e:
        print(f"Error importing module {module_name}: {e}")
    except Exception as e:
        print(f"Error processing module {module_name}: {e}")

def display_module_functions(module_name):
    """Display functions and their docstrings from a module"""
    try:
        # Import the module
        module = importlib.import_module(module_name)
        
        # Find all functions
        functions = inspect.getmembers(module, inspect.isfunction)
        
        # Filter functions defined in this module (not imported)
        functions = [(name, func) for name, func in functions if func.__module__ == module_name]
        
        # Print info for each function
        for name, func in functions:
            if name.startswith('_'):  # Skip private functions
                continue
                
            print(f"\n## {name}")
            
            # Get signature
            sig = inspect.signature(func)
            print(f"Signature: {name}{sig}")
            
            # Get docstring
            doc = inspect.getdoc(func)
            if doc:
                print(f"Docstring: {doc}")
            else:
                print("No docstring available")
    except ImportError as e:
        print(f"Error importing module {module_name}: {e}")
    except Exception as e:
        print(f"Error processing module {module_name}: {e}")

def view_function_source(module_name, function_name):
    """Display the source code of a function"""
    try:
        # Import the module
        module = importlib.import_module(module_name)
        
        # Get the function
        func = getattr(module, function_name)
        
        # Get the source code
        source = inspect.getsource(func)
        
        # Print the source code
        from IPython.display import display, Markdown
        display(Markdown(f"```python\n{source}\n```"))
    except ImportError as e:
        print(f"Error importing module {module_name}: {e}")
    except AttributeError:
        print(f"Function {function_name} not found in module {module_name}")
    except Exception as e:
        print(f"Error processing function {function_name}: {e}")

## Check Bonsai Installation

Let's verify that the Bonsai v3 module is available for import:

In [ ]:
try:
    from utils.bonsaitree.bonsaitree import v3
    print("✅ Successfully imported Bonsai v3 module")
except ImportError as e:
    print(f"❌ Failed to import Bonsai v3 module: {e}")
    print("This lab requires access to the Bonsai v3 codebase.")
    print("Make sure you've properly set up your environment with the Bonsai repository.")

## Lab 22: Interpreting Results and Confidence Measures

Genetic genealogy involves making inferences about relationships based on DNA data. However, these inferences come with varying degrees of certainty. Understanding the confidence levels and statistical significance of relationship predictions is crucial for proper interpretation of results.

In this lab, we'll examine how Bonsai v3 computes confidence measures and helps users interpret the reliability of predictions. We'll explore:

1. **Likelihood-Based Inference**: How Bonsai uses likelihood functions to assess the probability of different relationship types
2. **Confidence Intervals**: Methods for calculating confidence bounds on relationship degree estimates
3. **Age-Based Constraints**: How age information adds constraints and confidence to relationship predictions
4. **Multiple Hypotheses Testing**: Comparing alternative relationship hypotheses
5. **Visualizing Uncertainty**: Techniques for communicating confidence levels to users

Throughout this lab, we'll utilize the actual functions and methods from the Bonsai v3 codebase that handle confidence measures and result interpretation.

## Part 1: Likelihood-Based Inference

At the core of Bonsai's relationship prediction is the computation of likelihoods - statistical measures of how well the observed data fits different relationship hypotheses. Let's explore how Bonsai calculates and interprets these likelihoods:

In [ ]:
# Import the necessary Bonsai modules
try:
    from utils.bonsaitree.bonsaitree.v3 import likelihoods
    from utils.bonsaitree.bonsaitree.v3 import pedigrees
    from utils.bonsaitree.bonsaitree.v3 import ibd
    from utils.bonsaitree.bonsaitree.v3 import druid
    from utils.bonsaitree.bonsaitree.v3.constants import GENOME_LENGTH

    print("✅ Successfully imported Bonsai v3 modules")
    
    # Examine the PwLogLike class for computing relationship likelihoods
    if hasattr(likelihoods, 'PwLogLike'):
        print("\nExamining PwLogLike class from likelihoods.py:")
        print(f"Documentation: {likelihoods.PwLogLike.__doc__}")
        
    # Examine DRUID's degree inference function
    if hasattr(druid, 'infer_degree_generalized_druid'):
        print("\nExamining DRUID inference function:")
        print(f"Documentation: {druid.infer_degree_generalized_druid.__doc__}")
        
    # Examine confidence interval function
    if hasattr(likelihoods, 'get_total_ibd_deg_lbd_pt_ubd'):
        print("\nExamining confidence interval function:")
        print(f"Documentation: {likelihoods.get_total_ibd_deg_lbd_pt_ubd.__doc__}")
    
except ImportError as e:
    print(f"❌ Failed to import Bonsai v3 modules: {e}")
    print("This lab requires access to the Bonsai v3 codebase.")
    print("Please ensure the Bonsai repository is properly set up in your environment.")

### 1.1 Understanding Likelihood Functions

Likelihood functions in Bonsai calculate how probable the observed genetic data is under different relationship hypotheses. Let's examine some key functions used for likelihood-based inference:

In [ ]:
# Examine the functions for calculating the log likelihood of IBD data
try:
    # Check if the functions exist
    if hasattr(likelihoods, 'get_log_total_ibd_pdf'):
        print("Examining get_log_total_ibd_pdf function:")
        view_source(likelihoods.get_log_total_ibd_pdf)
    
    if hasattr(likelihoods, 'get_total_ibd_deg_lbd_pt_ubd'):
        print("\nExamining get_total_ibd_deg_lbd_pt_ubd function:")
        view_source(likelihoods.get_total_ibd_deg_lbd_pt_ubd)

    # Look for functions in point_predictor module
    if hasattr(point_predictor, 'point_predictions'):
        print("\nExamining point_predictions function:")
        view_source(point_predictor.point_predictions)
        
    # If we have the DRUID module available (for inference)
    try:
        from utils.bonsaitree.bonsaitree.v3 import druid
        print("\nExamining DRUID inference functions:")
        if hasattr(druid, 'infer_degree_generalized_druid'):
            view_source(druid.infer_degree_generalized_druid)
    except ImportError:
        print("\nDRUID module not available for examination")
        
except Exception as e:
    print(f"Error examining functions: {e}")

# Let's create a simplified explanation of how likelihood functions work
print("\n--- Simplified Explanation of Likelihood Functions ---")
print("""
Bonsai's likelihood functions calculate how well observed IBD data fits different relationship hypotheses:

1. For each possible relationship (parent-child, siblings, cousins, etc.):
   - Calculate the expected distribution of IBD sharing
   - Compare the observed IBD sharing to this distribution
   - Compute a likelihood score (higher = better fit)

2. The log-likelihood (LL) is used instead of raw likelihood for numerical stability:
   - LL(relationship | data) = log(P(data | relationship))
   - Higher log-likelihood indicates better fit

3. For complex pedigrees:
   - The total likelihood is computed from all pairwise relationships
   - Pedigree structures with higher overall likelihood are preferred
   
4. Confidence measures:
   - Relative likelihood differences indicate confidence
   - Larger differences between best and second-best hypothesis = higher confidence
""")

### 1.2 Working with Likelihood Scores

Now let's create a practical example to demonstrate how to work with likelihood scores and interpret them. We'll simulate some IBD data for different relationships and calculate the likelihoods:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
from scipy import stats
import math

# Set random seed for reproducibility
np.random.seed(42)
random.seed(42)

# Function to simulate IBD data for different relationships
def simulate_ibd_data(relationship_type, num_segments=None, add_noise=True):
    """
    Simulate IBD data for different relationship types.
    
    Args:
        relationship_type: Relationship type (parent-child, siblings, cousins, etc.)
        num_segments: Number of segments to simulate (if None, uses reasonable default)
        add_noise: Whether to add random noise to make it more realistic
        
    Returns:
        Dictionary with IBD data including total_cm and segments
    """
    # Set expected total cM sharing for common relationships (based on empirical averages)
    relationship_params = {
        'parent-child': {'mean_cm': 3400, 'std_cm': 100, 'default_segments': 23},
        'full-siblings': {'mean_cm': 2550, 'std_cm': 180, 'default_segments': 40},
        'half-siblings': {'mean_cm': 1700, 'std_cm': 160, 'default_segments': 30},
        '1st-cousins': {'mean_cm': 850, 'std_cm': 120, 'default_segments': 20},
        '2nd-cousins': {'mean_cm': 212, 'std_cm': 70, 'default_segments': 10},
        '3rd-cousins': {'mean_cm': 53, 'std_cm': 30, 'default_segments': 5},
        'unrelated': {'mean_cm': 10, 'std_cm': 10, 'default_segments': 1}
    }
    
    # Check if relationship type is valid
    if relationship_type not in relationship_params:
        raise ValueError(f"Unknown relationship type: {relationship_type}")
    
    # Get parameters for the relationship
    params = relationship_params[relationship_type]
    
    # Determine number of segments
    if num_segments is None:
        num_segments = params['default_segments']
    
    # Generate total cM with noise if requested
    if add_noise:
        total_cm = np.random.normal(params['mean_cm'], params['std_cm'])
        total_cm = max(0, total_cm)  # Ensure non-negative
    else:
        total_cm = params['mean_cm']
    
    # Generate segments
    segments = []
    remaining_cm = total_cm
    
    for i in range(num_segments - 1):
        # Each segment gets a random proportion of the remaining cM
        segment_cm = remaining_cm * random.random() * 0.3  # Up to 30% of remaining
        
        # Only include segments >= 7 cM (typical minimum threshold)
        if segment_cm >= 7:
            segments.append({
                'chromosome': str(random.randint(1, 22)),  # Random chromosome
                'start_pos': random.randint(1000000, 100000000),  # Random start position
                'end_pos': 0,  # Will be set below
                'cm': segment_cm,
                'snps': int(segment_cm * 70)  # Approximate SNP count
            })
            
            # Set end position based on cM length (very rough approximation)
            segments[-1]['end_pos'] = segments[-1]['start_pos'] + int(segment_cm * 1000000)
        
        remaining_cm -= segment_cm
    
    # Add the last segment with remaining cM
    if remaining_cm >= 7:
        segments.append({
            'chromosome': str(random.randint(1, 22)),
            'start_pos': random.randint(1000000, 100000000),
            'end_pos': 0,
            'cm': remaining_cm,
            'snps': int(remaining_cm * 70)
        })
        segments[-1]['end_pos'] = segments[-1]['start_pos'] + int(remaining_cm * 1000000)
    
    # Calculate actual total cM (might differ from target due to thresholds)
    actual_total_cm = sum(segment['cm'] for segment in segments)
    
    return {
        'total_cm': actual_total_cm,
        'num_segments': len(segments),
        'segments': segments,
        'true_relationship': relationship_type
    }

# Create simulated data for different relationships
relationships = ['parent-child', 'full-siblings', 'half-siblings', 
                '1st-cousins', '2nd-cousins', '3rd-cousins', 'unrelated']

simulated_data = {}
for rel in relationships:
    # Create 5 samples for each relationship type
    simulated_data[rel] = [simulate_ibd_data(rel) for _ in range(5)]

# Display summary of the simulated data
print("Simulated IBD Data Summary:")
print("-" * 70)
print(f"{'Relationship':<15} {'Total cM (Mean ± SD)':<25} {'Segments (Mean ± SD)':<20}")
print("-" * 70)

for rel in relationships:
    # Calculate statistics
    total_cm_values = [data['total_cm'] for data in simulated_data[rel]]
    segment_counts = [data['num_segments'] for data in simulated_data[rel]]
    
    mean_cm = np.mean(total_cm_values)
    std_cm = np.std(total_cm_values)
    mean_segments = np.mean(segment_counts)
    std_segments = np.std(segment_counts)
    
    print(f"{rel:<15} {mean_cm:>8.1f} ± {std_cm:<10.1f} {mean_segments:>8.1f} ± {std_segments:<8.1f}")

# Create visualization of the simulated data
plt.figure(figsize=(12, 6))

# Create a scatter plot of total cM vs number of segments
for rel in relationships:
    x = [data['total_cm'] for data in simulated_data[rel]]
    y = [data['num_segments'] for data in simulated_data[rel]]
    plt.scatter(x, y, alpha=0.7, label=rel)

plt.xlabel('Total cM Shared')
plt.ylabel('Number of Segments')
plt.title('IBD Sharing by Relationship Type')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Create a simplified likelihood calculation function 
def calculate_likelihood_scores(ibd_data):
    """
    Calculate simplified likelihood scores for different relationship hypotheses.
    
    This is a simplified implementation to demonstrate the concept when Bonsai's
    actual implementation is not available.
    
    Args:
        ibd_data: Dictionary with IBD data including total_cm and num_segments
        
    Returns:
        Dictionary with log-likelihood scores for each relationship type
    """
    # Expected values for different relationships
    relationship_params = {
        'parent-child': {'mean_cm': 3400, 'std_cm': 100},
        'full-siblings': {'mean_cm': 2550, 'std_cm': 180},
        'half-siblings': {'mean_cm': 1700, 'std_cm': 160},
        '1st-cousins': {'mean_cm': 850, 'std_cm': 120},
        '2nd-cousins': {'mean_cm': 212, 'std_cm': 70},
        '3rd-cousins': {'mean_cm': 53, 'std_cm': 30},
        'unrelated': {'mean_cm': 10, 'std_cm': 10}
    }
    
    # Get the observed total cM
    total_cm = ibd_data['total_cm']
    
    # Calculate log-likelihood for each relationship
    log_likelihoods = {}
    
    for rel, params in relationship_params.items():
        # Calculate log-likelihood using normal distribution as a simplified model
        # Real Bonsai uses more sophisticated models
        log_like = stats.norm.logpdf(total_cm, loc=params['mean_cm'], scale=params['std_cm'])
        
        # Store the log-likelihood
        log_likelihoods[rel] = log_like
    
    return log_likelihoods

# Calculate normalized likelihoods (for comparing relationships)
def normalize_log_likelihoods(log_likelihoods):
    """
    Normalize log-likelihoods to get posterior probabilities.
    This assumes equal priors for all relationships.
    
    Args:
        log_likelihoods: Dictionary with log-likelihood values
        
    Returns:
        Dictionary with normalized probabilities
    """
    # Convert to linear space
    linear_likelihoods = {rel: math.exp(ll) for rel, ll in log_likelihoods.items()}
    
    # Calculate sum for normalization
    total = sum(linear_likelihoods.values())
    
    # Normalize
    if total > 0:
        normalized = {rel: val/total for rel, val in linear_likelihoods.items()}
    else:
        # Handle numerical underflow
        max_ll = max(log_likelihoods.values())
        adjusted_ll = {rel: ll - max_ll for rel, ll in log_likelihoods.items()}
        linear_adj = {rel: math.exp(ll) for rel, ll in adjusted_ll.items()}
        total_adj = sum(linear_adj.values())
        normalized = {rel: val/total_adj for rel, val in linear_adj.items()}
    
    return normalized

# Demonstrate likelihood calculation for an example
example_relationship = 'half-siblings'
example_data = simulated_data[example_relationship][0]  # Take the first example

print(f"\nLikelihood Analysis for a {example_relationship} relationship:")
print(f"Total cM: {example_data['total_cm']:.1f}")
print(f"Segments: {example_data['num_segments']}")

# Calculate log-likelihoods
log_likelihoods = calculate_likelihood_scores(example_data)

# Sort by likelihood (highest first)
sorted_ll = sorted(log_likelihoods.items(), key=lambda x: x[1], reverse=True)

print("\nLog-Likelihood Scores:")
for rel, ll in sorted_ll:
    print(f"{rel:<15}: {ll:.2f}")

# Calculate normalized probabilities
normalized = normalize_log_likelihoods(log_likelihoods)
sorted_normalized = sorted(normalized.items(), key=lambda x: x[1], reverse=True)

print("\nNormalized Probabilities:")
for rel, prob in sorted_normalized:
    print(f"{rel:<15}: {prob:.4f} ({prob*100:.1f}%)")

# Calculate confidence as the ratio between top two probabilities
if len(sorted_normalized) >= 2:
    top_rel, top_prob = sorted_normalized[0]
    second_rel, second_prob = sorted_normalized[1]
    confidence_ratio = top_prob / second_prob if second_prob > 0 else float('inf')
    
    print(f"\nConfidence Ratio: {confidence_ratio:.2f}")
    if confidence_ratio > 100:
        confidence_level = "Very High"
    elif confidence_ratio > 10:
        confidence_level = "High"
    elif confidence_ratio > 2:
        confidence_level = "Moderate"
    else:
        confidence_level = "Low"
    
    print(f"Confidence Level: {confidence_level}")
    print(f"Top relationship is {confidence_ratio:.1f}x more likely than second-best hypothesis")

# Create a plot of the likelihoods
plt.figure(figsize=(10, 6))
relationships = [rel for rel, _ in sorted_normalized]
probabilities = [prob for _, prob in sorted_normalized]

plt.bar(relationships, probabilities, color='skyblue')
plt.xlabel('Relationship Type')
plt.ylabel('Probability')
plt.title('Relationship Probability Distribution')
plt.xticks(rotation=45)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

### 1.3 Using PwLogLike for Relationship Inference

Now let's try to use Bonsai's actual `PwLogLike` class for relationship inference if it's available:

In [ ]:
# Use Bonsai's PwLogLike class for relationship inference
try:
    from utils.bonsaitree.bonsaitree.v3.likelihoods import PwLogLike
    from utils.bonsaitree.bonsaitree.v3.ibd import IBDSegment

    print("✅ Using Bonsai v3's actual PwLogLike class for relationship inference")
    
    # First, convert our simulated IBD data to Bonsai's expected format for unphased IBD segments
    # Format: [[id1, id2, chromosome, start_bp, end_bp, is_full_ibd, seg_len_cm],...]
    def convert_to_bonsai_unphased_segments(ibd_data, id1=1, id2=2):
        """Convert simulated IBD data to Bonsai's unphased segment format"""
        bonsai_segments = []
        
        for segment in ibd_data['segments']:
            # Create unphased segment record
            unphased_seg = [
                id1,                      # id1 
                id2,                      # id2
                segment['chromosome'],    # chromosome
                segment['start_pos'],     # start_bp
                segment['end_pos'],       # end_bp
                False,                    # is_full_ibd (defaults to False for simplicity)
                segment['cm']             # seg_len_cm
            ]
            bonsai_segments.append(unphased_seg)
                
        return bonsai_segments
    
    # Create sample bioinfo for the two individuals
    bio_info = [
        {'genotype_id': 1, 'age': 50, 'sex': 'M'},
        {'genotype_id': 2, 'age': 30, 'sex': 'F'}
    ]
    
    # Convert example data to Bonsai format
    unphased_segments = convert_to_bonsai_unphased_segments(example_data)
    print(f"Created {len(unphased_segments)} Bonsai unphased segments from example data")
    
    # Create a PwLogLike object with our sample data
    pw_log_like = PwLogLike(
        bio_info=bio_info,
        unphased_ibd_seg_list=unphased_segments
    )
    
    # Define relationship tuples to test
    relationships_to_test = {
        'parent-child': (0, 1, 1),      # (up, down, num_ancestors)
        'full-siblings': (1, 1, 2),
        'half-siblings': (1, 1, 1),
        '1st-cousins': (2, 2, 1),
        '2nd-cousins': (3, 3, 1),
        '3rd-cousins': (4, 4, 1),
        'unrelated': None               # None indicates no relationship
    }
    
    # Get log-likelihoods for each relationship
    rel_scores = {}
    for rel_name, rel_tuple in relationships_to_test.items():
        try:
            # Get genetic component of the likelihood
            gen_ll = pw_log_like.get_pw_gen_ll(
                node1=1, 
                node2=2, 
                rel_tuple=rel_tuple
            )
            
            # Get age component of the likelihood
            age_ll = pw_log_like.get_pw_age_ll(
                node1=1,
                node2=2,
                rel_tuple=rel_tuple
            )
            
            # Total log-likelihood is the sum of genetic and age components
            total_ll = gen_ll + age_ll
            rel_scores[rel_name] = total_ll
        except Exception as e:
            print(f"Error calculating likelihood for {rel_name}: {e}")
            # Fall back to genetic component only
            try:
                gen_ll = pw_log_like.get_pw_gen_ll(
                    node1=1, 
                    node2=2, 
                    rel_tuple=rel_tuple
                )
                rel_scores[rel_name] = gen_ll
            except Exception as e:
                print(f"Error calculating genetic likelihood for {rel_name}: {e}")
                rel_scores[rel_name] = float('-inf')
    
    # Display the log-likelihood scores
    print("\nBonsai PwLogLike Scores:")
    for rel, score in sorted(rel_scores.items(), key=lambda x: x[1], reverse=True):
        print(f"{rel:<15}: {score:.2f}")
    
    # Find most likely relationship
    most_likely_rel = max(rel_scores.items(), key=lambda x: x[1])[0]
    print(f"\nMost likely relationship: {most_likely_rel}")
    
    # Calculate confidence measures
    # Method 1: Use likelihood ratio between best and second-best hypothesis
    sorted_scores = sorted(rel_scores.items(), key=lambda x: x[1], reverse=True)
    if len(sorted_scores) >= 2:
        top_rel, top_ll = sorted_scores[0]
        second_rel, second_ll = sorted_scores[1]
        # Convert log-likelihoods to likelihood ratio
        ll_ratio = np.exp(top_ll - second_ll)
        
        # Method 2: Calculate degree confidence interval
        if hasattr(likelihoods, 'get_total_ibd_deg_lbd_pt_ubd'):
            try:
                # Get relationship degree
                rel_tuple = relationships_to_test[top_rel]
                if rel_tuple is not None:
                    # Use Bonsai's confidence interval function if the relationship has ancestors
                    a = rel_tuple[2]  # number of ancestors
                    total_cm = sum(segment['cm'] for segment in example_data['segments'])
                    
                    lbd, mlm, ubd = likelihoods.get_total_ibd_deg_lbd_pt_ubd(
                        a=a,
                        L=total_cm,
                        condition=True  # Most IBD detectors condition on observing at least one segment
                    )
                    
                    print(f"\nDegree confidence interval (95%):")
                    print(f"Lower bound: {lbd}")
                    print(f"Point estimate: {mlm}")
                    print(f"Upper bound: {ubd}")
            except Exception as e:
                print(f"Error calculating confidence interval: {e}")
        
        # Interpret confidence
        print(f"\nConfidence measure (likelihood ratio):")
        print(f"{top_rel} is {ll_ratio:.1f}x more likely than {second_rel}")
        
        if ll_ratio > 100:
            confidence_level = "Very High"
        elif ll_ratio > 10:
            confidence_level = "High"
        elif ll_ratio > 2:
            confidence_level = "Moderate"
        else:
            confidence_level = "Low"
        
        print(f"Confidence level: {confidence_level}")
        
except ImportError as e:
    print(f"❌ Failed to import Bonsai classes: {e}")
    print("Falling back to simplified implementation since Bonsai is not available")
    
    # Falling back to simplified implementation, which we'll only run if Bonsai is unavailable
    # Calculate simplified likelihood
    simple_ll = calculate_likelihood_scores(example_data)
    simple_normalized = normalize_log_likelihoods(simple_ll)
    
    # Display results
    print("\nResults using simplified model (only because Bonsai is unavailable):")
    for rel, prob in sorted(simple_normalized.items(), key=lambda x: x[1], reverse=True)[:3]:
        print(f"{rel:<15}: {prob:.4f} ({prob*100:.1f}%)")

In [ ]:
# Implementing confidence interval calculation using Bonsai's methods
try:
    # First try to use Bonsai's actual implementation
    import numpy as np
    from utils.bonsaitree.bonsaitree.v3 import likelihoods
    from utils.bonsaitree.bonsaitree.v3.constants import GENOME_LENGTH, DEG_CI_ALPHA
    
    print("✅ Using Bonsai v3's methods for confidence interval calculation")
    
    # Function to demonstrate Bonsai's confidence interval calculation
    def calculate_bonsai_confidence_interval(L, a=1, condition=True, alpha=DEG_CI_ALPHA):
        """
        Calculate a confidence interval for relationship degree using Bonsai's method.
        
        Args:
            L: Total IBD length in cM
            a: Number of common ancestors (default: 1)
            condition: Whether to condition on observing at least one segment (default: True)
            alpha: Confidence level (default: Bonsai's DEG_CI_ALPHA)
            
        Returns:
            Dictionary with confidence interval information
        """
        # Use Bonsai's actual function for confidence interval calculation
        lbd, mlm, ubd = likelihoods.get_total_ibd_deg_lbd_pt_ubd(
            a=a,
            L=L,
            condition=condition,
            alpha=alpha
        )
        
        return {
            'lower_bound': lbd,
            'point_estimate': mlm,
            'upper_bound': ubd,
            'confidence_level': 1-alpha
        }
    
    # Use our example data from the previous section
    total_cm = sum(segment['cm'] for segment in example_data['segments'])
    print(f"Total IBD: {total_cm:.1f} cM")
    
    # Calculate confidence interval using Bonsai's method
    rel_tuple = relationships_to_test.get(example_relationship)
    if rel_tuple is not None:
        a = rel_tuple[2]  # Number of common ancestors
        
        conf_interval = calculate_bonsai_confidence_interval(total_cm, a=a)
        
        # Display the results
        print(f"\nRelationship Confidence Interval ({(1-DEG_CI_ALPHA)*100:.0f}%):")
        print(f"Lower bound: {conf_interval['lower_bound']} meioses")
        print(f"Point estimate: {conf_interval['point_estimate']} meioses")
        print(f"Upper bound: {conf_interval['upper_bound']} meioses")
        
        # Convert meioses to relationship degrees for common relationships
        meioses_to_rel = {
            1: "parent-child",
            2: "full-siblings or grandparent",
            3: "avuncular or half-siblings",
            4: "1st-cousins",
            5: "1st-cousins-once-removed",
            6: "2nd-cousins",
            8: "3rd-cousins",
            10: "4th-cousins"
        }
        
        # Print degree interpretation
        print("\nDegree interpretation:")
        point_estimate_rel = meioses_to_rel.get(int(conf_interval['point_estimate']), f"{int(conf_interval['point_estimate'])}-degree")
        print(f"Most likely: {point_estimate_rel}")
        
        # List all plausible relationships within confidence interval
        plausible_rels = []
        for m in range(int(conf_interval['lower_bound']), int(conf_interval['upper_bound'])+1):
            rel = meioses_to_rel.get(m, f"{m}-degree")
            plausible_rels.append(rel)
        
        print(f"Plausible relationships: {', '.join(plausible_rels)}")
        
    # Visualize the confidence interval on a likelihood curve
    # Try to use Bonsai's actual get_log_total_ibd_pdf function
    if hasattr(likelihoods, 'get_log_total_ibd_pdf'):
        plt.figure(figsize=(12, 6))
        
        # Calculate likelihoods over a range of meioses values
        meioses_range = np.arange(1, 15)
        a = rel_tuple[2] if rel_tuple is not None else 1
        
        # Get log-likelihoods using Bonsai's function
        log_pdfs = likelihoods.get_log_total_ibd_pdf(
            a=a,
            m=meioses_range,
            L=total_cm,
            condition=True
        )
        
        # Normalize the log PDFs to get the posterior distribution
        log_pdfs_norm = log_pdfs - np.log(np.sum(np.exp(log_pdfs)))
        # Convert to probabilities
        probs = np.exp(log_pdfs_norm)
        
        # Plot the probability distribution
        plt.plot(meioses_range, probs, 'o-', color='blue', markersize=8)
        
        # Highlight confidence interval
        if rel_tuple is not None:
            min_y = 0
            max_y = max(probs) * 1.1
            plt.fill_between([conf_interval['lower_bound'], conf_interval['upper_bound']], 
                            min_y, max_y, color='lightblue', alpha=0.3)
            
            # Annotate the plot
            plt.text(conf_interval['point_estimate'], max(probs),
                     f"Point estimate: {conf_interval['point_estimate']:.1f}",
                     ha='center', va='bottom')
        
        # Add relationship labels
        for m in meioses_range:
            rel = meioses_to_rel.get(m, "")
            if rel:
                plt.annotate(rel, (m, probs[m-1]), 
                            xytext=(0, 10), textcoords='offset points',
                            ha='center', rotation=45)
        
        plt.xlabel('Number of Meioses (m)')
        plt.ylabel('Posterior Probability')
        plt.title('Relationship Inference Confidence')
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()
    
    # Note on how Bonsai calculates confidence intervals
    print("\nHow Bonsai Calculates Confidence Intervals:")
    print("""
Bonsai v3 uses several methods to calculate confidence intervals:

1. Likelihood-based confidence intervals:
   - Uses the highest posterior density (HPD) approach
   - Includes all meiosis values that together contain (1-alpha)% of the 
     posterior probability mass
   - The point estimate is the meiosis count with the highest likelihood

2. Statistical properties:
   - The width of the confidence interval reflects uncertainty in the estimate
   - Wider intervals indicate more uncertainty
   - Narrow intervals indicate confident predictions

3. Implementation steps:
   - Compute the posterior probability distribution over meiosis values
   - Sort meiosis values by their posterior probability (highest first)
   - Include meiosis values until the cumulative probability exceeds the threshold
   - Find the minimum and maximum meiosis in the included set
""")

except (ImportError, AttributeError) as e:
    print(f"❌ Error using Bonsai's confidence interval methods: {e}")
    print("Implementing simplified confidence interval calculation")
    
    # Only implement this simplified version if Bonsai's actual implementation is unavailable
    import numpy as np
    from scipy import stats
    import matplotlib.pyplot as plt

    # Function to calculate confidence intervals for relationship degree
    def calculate_confidence_interval(log_likelihoods, confidence_level=0.95):
        """
        Calculate a confidence interval for the relationship degree based on likelihood scores.
        
        This is a simplified implementation that uses the likelihood ratio test approach.
        
        Args:
            log_likelihoods: Dictionary of log-likelihood scores for different relationships
            confidence_level: Desired confidence level (e.g., 0.95 for 95% confidence)
            
        Returns:
            Dictionary with confidence interval information
        """
        # Map relationship types to numerical degrees for simplicity
        degree_mapping = {
            'parent-child': 1,
            'full-siblings': 1,
            'half-siblings': 1.5,
            '1st-cousins': 2,
            '2nd-cousins': 3,
            '3rd-cousins': 4,
            'unrelated': 10
        }
        
        # Get the best relationship and its log-likelihood
        best_rel = max(log_likelihoods.items(), key=lambda x: x[1])[0]
        best_ll = log_likelihoods[best_rel]
        
        # Calculate critical value for likelihood ratio test
        # For degree inference, a chi-square critical value is often used
        # with 1 degree of freedom
        critical_value = stats.chi2.ppf(confidence_level, 1)
        
        # Find relationships that are not significantly different from the best
        plausible_rels = []
        for rel, ll in log_likelihoods.items():
            # Twice the log-likelihood difference is approximately chi-square distributed
            lr_stat = 2 * (best_ll - ll)
            
            if lr_stat <= critical_value:
                plausible_rels.append((rel, degree_mapping.get(rel, 0), ll))
        
        # Sort by degree
        plausible_rels.sort(key=lambda x: x[1])
        
        # Extract min and max degrees
        min_degree = min(degree for _, degree, _ in plausible_rels)
        max_degree = max(degree for _, degree, _ in plausible_rels)
        
        return {
            'best_relationship': best_rel,
            'best_degree': degree_mapping.get(best_rel, 0),
            'min_degree': min_degree,
            'max_degree': max_degree,
            'confidence_level': confidence_level,
            'plausible_relationships': [rel for rel, _, _ in plausible_rels]
        }

    # Using our example data from the previous section
    conf_interval = calculate_confidence_interval(log_likelihoods, confidence_level=0.95)

    # Display the results
    print(f"Relationship Confidence Interval (95%):")
    print(f"Best estimate: {conf_interval['best_relationship']} (Degree {conf_interval['best_degree']})")
    print(f"Degree range: {conf_interval['min_degree']} to {conf_interval['max_degree']}")
    print(f"Plausible relationships: {', '.join(conf_interval['plausible_relationships']}")

    # Visualize the confidence interval
    plt.figure(figsize=(10, 6))

    # Plot degree vs negative log-likelihood
    degrees = []
    neg_ll = []
    rel_labels = []

    for rel, ll in log_likelihoods.items():
        if rel in degree_mapping:
            degrees.append(degree_mapping[rel])
            neg_ll.append(-ll)  # Negative log-likelihood
            rel_labels.append(rel)

    # Sort by degree
    sorted_indices = np.argsort(degrees)
    degrees = [degrees[i] for i in sorted_indices]
    neg_ll = [neg_ll[i] for i in sorted_indices]
    rel_labels = [rel_labels[i] for i in sorted_indices]

    # Plot
    plt.plot(degrees, neg_ll, 'o-', color='blue', markersize=8)

    # Highlight confidence interval
    min_y = min(neg_ll) - 5
    max_y = max(neg_ll) + 5
    plt.fill_between([conf_interval['min_degree'], conf_interval['max_degree']], 
                    min_y, max_y, color='lightblue', alpha=0.3)

    # Add relationship labels
    for i, txt in enumerate(rel_labels):
        plt.annotate(txt, (degrees[i], neg_ll[i]), 
                    xytext=(5, 5), textcoords='offset points')

    plt.xlabel('Relationship Degree')
    plt.ylabel('Negative Log-Likelihood')
    plt.title('Relationship Degree Confidence Interval (95%)')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

## Part 3: Age-Based Constraints in Relationship Inference

Age information provides valuable constraints that can significantly improve relationship inference. Age differences between individuals impose logical constraints on possible relationships. For example:

- Parent-child relationships require approximately 15-45 years age difference
- Sibling relationships typically involve individuals with age differences less than 30 years
- First cousin relationships typically involve similar ages, or age differences up to 30 years

Bonsai v3 can incorporate age information to refine likelihood calculations and improve confidence in relationship predictions. Let's explore how:

In [ ]:
# Demonstrate how Bonsai uses age constraints for relationship inference
try:
    # First try to use Bonsai's actual implementation
    from utils.bonsaitree.bonsaitree.v3.likelihoods import get_age_log_like, get_age_mean_std_for_rel_tuple

    print("✅ Using Bonsai v3's actual age-based constraint functions")
    
    # Create a function to demonstrate how Bonsai incorporates age information
    def calculate_age_constrained_likelihoods_bonsai(genetic_log_likelihoods, id1_age, id2_age, relationships_to_test):
        """
        Incorporate age information into relationship likelihoods using Bonsai's actual functions.
        
        Args:
            genetic_log_likelihoods: Dictionary of log-likelihoods from genetic data
            id1_age: Age of person 1
            id2_age: Age of person 2
            relationships_to_test: Dictionary mapping relationship names to relationship tuples
            
        Returns:
            Dictionary of updated log-likelihoods
        """
        updated_log_likelihoods = {}
        
        # For each relationship
        for rel_name, rel_tuple in relationships_to_test.items():
            if rel_name in genetic_log_likelihoods:
                genetic_ll = genetic_log_likelihoods[rel_name]
                
                # Get age log-likelihood using Bonsai's function
                age_ll = get_age_log_like(
                    age1=id1_age,
                    age2=id2_age,
                    rel_tuple=rel_tuple
                )
                
                # Add log probabilities (equivalent to multiplying in linear space)
                updated_ll = genetic_ll + age_ll
                updated_log_likelihoods[rel_name] = updated_ll
        
        return updated_log_likelihoods
    
    # Get the age distributions for different relationships from Bonsai
    def plot_bonsai_age_distributions(relationships_to_test):
        """Plot age difference distributions for different relationships using Bonsai's model"""
        plt.figure(figsize=(12, 6))
        
        age_diffs = range(-60, 60)
        
        for rel_name, rel_tuple in relationships_to_test.items():
            if rel_tuple is not None:  # Skip 'unrelated'
                # Get mean and standard deviation for the relationship
                mean, std = get_age_mean_std_for_rel_tuple(rel_tuple)
                
                # Calculate probability for each age difference
                probs = [np.exp(scipy.stats.norm.logpdf(diff, loc=mean, scale=std)) for diff in age_diffs]
                
                plt.plot(age_diffs, probs, label=rel_name)
        
        plt.xlabel('Age Difference (years)')
        plt.ylabel('Probability Density')
        plt.title('Bonsai Age Difference Distributions by Relationship Type')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()
    
    # Plot the age distributions using Bonsai's model
    plot_bonsai_age_distributions(relationships_to_test)
    
    # Demonstrate how age information affects relationship predictions
    # Create different scenarios with different age differences
    age_scenarios = {
        'scenario1': {'id1_age': 70, 'id2_age': 45, 'description': 'Typical parent-child age difference'},
        'scenario2': {'id1_age': 45, 'id2_age': 42, 'description': 'Typical sibling age difference'},
        'scenario3': {'id1_age': 45, 'id2_age': 20, 'description': 'Typical avuncular age difference'},
        'scenario4': {'id1_age': 45, 'id2_age': 45, 'description': 'Same age (could be cousins or siblings)'}
    }
    
    # Get genetic log-likelihoods from our previous example
    genetic_ll = rel_scores
    
    # Create a table to compare results
    results = []
    
    for scenario_name, scenario in age_scenarios.items():
        # Get age-constrained likelihoods using Bonsai's function
        age_constrained_ll = calculate_age_constrained_likelihoods_bonsai(
            genetic_log_likelihoods=genetic_ll,
            id1_age=scenario['id1_age'],
            id2_age=scenario['id2_age'],
            relationships_to_test=relationships_to_test
        )
        
        # Find best relationship with genetic data only
        best_rel_genetic = max(genetic_ll.items(), key=lambda x: x[1])[0]
        
        # Find best relationship with age constraints
        best_rel_constrained = max(age_constrained_ll.items(), key=lambda x: x[1])[0]
        
        # Calculate likelihood ratios for top two relationships
        # For genetic only
        genetic_sorted = sorted(genetic_ll.items(), key=lambda x: x[1], reverse=True)
        genetic_ratio = np.exp(genetic_sorted[0][1] - genetic_sorted[1][1]) if len(genetic_sorted) > 1 else float('inf')
        
        # For age constrained
        constrained_sorted = sorted(age_constrained_ll.items(), key=lambda x: x[1], reverse=True)
        constrained_ratio = np.exp(constrained_sorted[0][1] - constrained_sorted[1][1]) if len(constrained_sorted) > 1 else float('inf')
        
        # Store results
        results.append({
            'scenario': scenario_name,
            'description': scenario['description'],
            'age_diff': scenario['id1_age'] - scenario['id2_age'],
            'genetic_best': best_rel_genetic,
            'constrained_best': best_rel_constrained,
            'genetic_ratio': genetic_ratio,
            'constrained_ratio': constrained_ratio,
        })
    
    # Display results as a table
    result_df = pd.DataFrame(results)
    print("\nEffect of Age Difference on Relationship Prediction using Bonsai:")
    print(result_df[['scenario', 'description', 'age_diff', 'genetic_best', 'constrained_best', 'genetic_ratio', 'constrained_ratio']])
    
    # Create visualization of genetic vs age+genetic for one scenario
    chosen_scenario = 'scenario1'  # Parent-child age difference
    scenario = age_scenarios[chosen_scenario]
    
    # Calculate constrained likelihoods for this scenario
    age_constrained_ll = calculate_age_constrained_likelihoods_bonsai(
        genetic_log_likelihoods=genetic_ll,
        id1_age=scenario['id1_age'],
        id2_age=scenario['id2_age'],
        relationships_to_test=relationships_to_test
    )
    
    # Convert log-likelihoods to normalized probabilities
    def normalize_log_likelihoods_bonsai(log_likelihoods):
        """Normalize log-likelihoods to get posterior probabilities"""
        # Convert to linear space
        linear_likelihoods = {rel: np.exp(ll) for rel, ll in log_likelihoods.items()}
        
        # Calculate sum for normalization
        total = sum(linear_likelihoods.values())
        
        # Normalize
        if total > 0:
            normalized = {rel: val/total for rel, val in linear_likelihoods.items()}
        else:
            # Handle numerical underflow
            max_ll = max(log_likelihoods.values())
            adjusted_ll = {rel: ll - max_ll for rel, ll in log_likelihoods.items()}
            linear_adj = {rel: np.exp(ll) for rel, ll in adjusted_ll.items()}
            total_adj = sum(linear_adj.values())
            normalized = {rel: val/total_adj for rel, val in linear_adj.items()}
        
        return normalized
    
    # Get normalized probabilities
    genetic_norm = normalize_log_likelihoods_bonsai(genetic_ll)
    age_constrained_norm = normalize_log_likelihoods_bonsai(age_constrained_ll)
    
    # Prepare for visualization
    rel_order = ['parent-child', 'full-siblings', 'half-siblings', 
                '1st-cousins', '2nd-cousins', '3rd-cousins', 'unrelated']
    
    # Only include relationships that exist in both dictionaries
    rel_order = [rel for rel in rel_order if rel in genetic_norm and rel in age_constrained_norm]
    
    genetic_probs = [genetic_norm.get(rel, 0) for rel in rel_order]
    constrained_probs = [age_constrained_norm.get(rel, 0) for rel in rel_order]
    
    # Create bar chart comparison
    plt.figure(figsize=(12, 6))
    x = np.arange(len(rel_order))
    width = 0.35
    
    plt.bar(x - width/2, genetic_probs, width, label='Genetic Data Only')
    plt.bar(x + width/2, constrained_probs, width, label='Genetic + Age Data')
    
    plt.xlabel('Relationship Type')
    plt.ylabel('Probability')
    plt.title(f'Impact of Age Information (Age Diff: {scenario["id1_age"] - scenario["id2_age"]} years)')
    plt.xticks(x, rel_order, rotation=45)
    plt.legend()
    plt.grid(axis='y', alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # Summary of how Bonsai uses age information
    print("\nHow Bonsai v3 Uses Age Information:")
    print("""
Bonsai v3 systematically incorporates age information in several ways:

1. Empirical Age Distribution Modeling:
   - Maintains a database of age differences for each relationship type
   - Models the distribution of age differences using normal distributions
   - Parameters estimated from empirical data

2. Age Likelihood Component:
   - Log-likelihood component for age: log(P(age_diff | relationship))
   - Combined with genetic component: total_ll = genetic_ll + age_ll
   - Mathematically principled Bayesian approach

3. Hard Constraints:
   - Some relationships naturally imply constraints (e.g., parents must be older than children)
   - Impossible age differences receive -infinity log-likelihood

4. Pedigree Consistency:
   - In complex pedigrees, ensures age consistency across the entire structure
   - Identifies and penalizes age-inconsistent pedigrees
""")
    
except (ImportError, AttributeError) as e:
    print(f"❌ Error using Bonsai's age constraint functions: {e}")
    print("Implementing simplified age-constraint calculation")
    
    # Only implement the simplified version if Bonsai's implementation is unavailable
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    import random
    from scipy import stats

    # Define typical age difference distributions for various relationships
    def get_age_difference_probability(relationship, age_diff):
        """
        Calculate probability of a given age difference for a specific relationship.
        
        Args:
            relationship: Relationship type
            age_diff: Age difference in years (can be negative)
        
        Returns:
            Probability density at the given age difference
        """
        # Parameters for distributions based on typical age differences
        rel_params = {
            'parent-child': {'mean': 30, 'std': 5, 'min': 15, 'max': 50, 'direction': 1},
            'full-siblings': {'mean': 0, 'std': 5, 'min': -30, 'max': 30, 'direction': 0},
            'half-siblings': {'mean': 0, 'std': 7, 'min': -40, 'max': 40, 'direction': 0},
            '1st-cousins': {'mean': 0, 'std': 8, 'min': -40, 'max': 40, 'direction': 0},
            '2nd-cousins': {'mean': 0, 'std': 10, 'min': -50, 'max': 50, 'direction': 0},
            '3rd-cousins': {'mean': 0, 'std': 12, 'min': -60, 'max': 60, 'direction': 0},
            'unrelated': {'mean': 0, 'std': 20, 'min': -100, 'max': 100, 'direction': 0}
        }
        
        # Check if relationship exists
        if relationship not in rel_params:
            return 0.0
        
        params = rel_params[relationship]
        
        # Apply directional constraint for parent-child
        if params['direction'] == 1 and age_diff < 0:
            return 0.0  # Parent must be older than child
        
        # Check if age difference is in allowed range
        if age_diff < params['min'] or age_diff > params['max']:
            return 0.0
        
        # Calculate probability using normal distribution
        prob = stats.norm.pdf(age_diff, loc=params['mean'], scale=params['std'])
        
        # Normalize to account for truncation
        norm_factor = stats.norm.cdf(params['max'], loc=params['mean'], scale=params['std']) - \
                    stats.norm.cdf(params['min'], loc=params['mean'], scale=params['std'])
        
        if norm_factor > 0:
            prob = prob / norm_factor
        
        return prob

    # Create a function to incorporate age information into relationship likelihoods
    def calculate_age_constrained_likelihoods(genetic_log_likelihoods, age_diff):
        """
        Incorporate age information into relationship likelihoods.
        
        Args:
            genetic_log_likelihoods: Dictionary of log-likelihoods from genetic data
            age_diff: Age difference in years (person1 age - person2 age)
            
        Returns:
            Dictionary of updated log-likelihoods
        """
        # Start with genetic log-likelihoods
        updated_log_likelihoods = {}
        
        # For each relationship
        for rel, genetic_ll in genetic_log_likelihoods.items():
            # Get age probability
            age_prob = get_age_difference_probability(rel, age_diff)
            
            # Convert to log space
            if age_prob > 0:
                age_log_prob = np.log(age_prob)
                
                # Add log probabilities (equivalent to multiplying in linear space)
                updated_ll = genetic_ll + age_log_prob
                updated_log_likelihoods[rel] = updated_ll
            else:
                # Age difference is impossible for this relationship
                updated_log_likelihoods[rel] = float('-inf')  # Effectively zero probability
        
        return updated_log_likelihoods

    # Create a function to visualize age distributions for different relationships
    def plot_age_diff_distributions():
        """Plot age difference distributions for different relationships"""
        relationships = ['parent-child', 'full-siblings', 'half-siblings', 
                        '1st-cousins', '2nd-cousins', '3rd-cousins']
        
        age_diffs = range(-60, 60)
        
        plt.figure(figsize=(12, 6))
        
        for rel in relationships:
            probs = [get_age_difference_probability(rel, diff) for diff in age_diffs]
            plt.plot(age_diffs, probs, label=rel)
        
        plt.xlabel('Age Difference (years)')
        plt.ylabel('Probability Density')
        plt.title('Age Difference Distributions by Relationship Type')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()

    # Plot age difference distributions
    plot_age_diff_distributions()

    # Demonstrate how age information affects relationship predictions
    # Example: Using our half-sibling example and varying age differences
    example_genetic_ll = log_likelihoods  # From our previous example

    # Try different age differences
    age_differences = [-40, -20, 0, 20, 40]

    # Create a table to compare results
    results = []

    for age_diff in age_differences:
        # Calculate age-constrained likelihoods
        age_constrained_ll = calculate_age_constrained_likelihoods(example_genetic_ll, age_diff)
        
        # Find best relationship
        best_rel = "none"
        best_ll = float('-inf')
        
        for rel, ll in age_constrained_ll.items():
            if ll > best_ll and ll > float('-inf'):
                best_rel = rel
                best_ll = ll
        
        # Store results
        results.append({
            'age_diff': age_diff,
            'best_relationship': best_rel,
            'log_likelihood': best_ll if best_ll > float('-inf') else "N/A"
        })

    # Display results as a table
    result_df = pd.DataFrame(results)
    print("\nEffect of Age Difference on Relationship Prediction (simplified model):")
    print(result_df)

    # Example of how age information is incorporated
    print("\nSimplified model of how age information can be incorporated:")
    print("1. Prior probabilities: Adjust relationship priors based on age differences")
    print("2. Likelihood modifiers: Multiply genetic likelihoods by age-based factors")
    print("3. Hard constraints: Rule out impossible relationships based on age")
    print("4. Pedigree consistency: Ensure the entire pedigree has age-consistent connections")

    # Example: Visualize how adding age constraints improves confidence
    # Using our half-sibling example and a specific age difference
    chosen_age_diff = 2  # Example: 2 years apart (consistent with siblings)

    # Calculate age-constrained likelihoods
    age_constrained_ll = calculate_age_constrained_likelihoods(example_genetic_ll, chosen_age_diff)

    # Calculate normalized probabilities for both
    genetic_only_norm = normalize_log_likelihoods(example_genetic_ll)
    age_constrained_norm = normalize_log_likelihoods(age_constrained_ll)

    # Prepare data for visualization
    rel_order = ['parent-child', 'full-siblings', 'half-siblings', 
                '1st-cousins', '2nd-cousins', '3rd-cousins', 'unrelated']

    genetic_only_probs = [genetic_only_norm.get(rel, 0) for rel in rel_order]
    age_constrained_probs = [age_constrained_norm.get(rel, 0) for rel in rel_order]

    # Create bar chart comparison
    plt.figure(figsize=(12, 6))
    x = np.arange(len(rel_order))
    width = 0.35

    plt.bar(x - width/2, genetic_only_probs, width, label='Genetic Data Only')
    plt.bar(x + width/2, age_constrained_probs, width, label='Genetic + Age Data')

    plt.xlabel('Relationship Type')
    plt.ylabel('Probability')
    plt.title(f'Impact of Age Information (Age Diff: {chosen_age_diff} years)')
    plt.xticks(x, rel_order, rotation=45)
    plt.legend()
    plt.grid(axis='y', alpha=0.3)
    plt.tight_layout()
    plt.show()

## Part 4: Multiple Hypothesis Testing in Relationship Inference

In genetic genealogy, we often need to compare multiple relationship hypotheses to determine the most likely connection between individuals. This is especially important when:

1. The genetic evidence alone is ambiguous between two or more relationship types
2. We have prior information about potential relationships from traditional genealogy
3. We need to evaluate whether specific individuals fit into a particular pedigree structure

Bonsai v3 implements systematic approaches to test multiple relationship hypotheses and determine which is most consistent with the observed genetic data.

In [ ]:
# Multiple Hypothesis Testing for Relationship Inference
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from scipy import stats
from scipy.special import logsumexp
import math

# Define a function to create specific relationship hypotheses
def create_relationship_hypothesis(relationship_type):
    """
    Create a formal relationship hypothesis structure.
    
    Args:
        relationship_type: Type of relationship (e.g., 'parent-child', 'full-siblings')
        
    Returns:
        Dictionary representing the relationship hypothesis
    """
    # Map of relationship types to degree and specific properties
    relationship_props = {
        'parent-child': {
            'degree': 1,
            'expected_total_cm': 3400,
            'expected_segments': 23,
            'cm_std': 100,
            'description': "Parent-child relationship"
        },
        'full-siblings': {
            'degree': 1,
            'expected_total_cm': 2550,
            'expected_segments': 40, 
            'cm_std': 180,
            'description': "Full siblings sharing both parents"
        },
        'half-siblings': {
            'degree': 1.5,
            'expected_total_cm': 1700,
            'expected_segments': 30,
            'cm_std': 160,
            'description': "Half siblings sharing one parent"
        },
        '1st-cousins': {
            'degree': 2,
            'expected_total_cm': 850,
            'expected_segments': 20,
            'cm_std': 120,
            'description': "First cousins sharing grandparents"
        },
        '2nd-cousins': {
            'degree': 3,
            'expected_total_cm': 212,
            'expected_segments': 10,
            'cm_std': 70,
            'description': "Second cousins sharing great-grandparents"
        },
        '3rd-cousins': {
            'degree': 4,
            'expected_total_cm': 53,
            'expected_segments': 5,
            'cm_std': 30,
            'description': "Third cousins sharing great-great-grandparents"
        },
        'unrelated': {
            'degree': 10,
            'expected_total_cm': 10,
            'expected_segments': 1,
            'cm_std': 10,
            'description': "No known relationship"
        }
    }
    
    if relationship_type not in relationship_props:
        raise ValueError(f"Unknown relationship type: {relationship_type}")
    
    # Create the hypothesis
    hypothesis = {
        'name': relationship_type,
        'degree': relationship_props[relationship_type]['degree'],
        'expected_ibd': relationship_props[relationship_type]['expected_total_cm'],
        'expected_segments': relationship_props[relationship_type]['expected_segments'],
        'std_dev': relationship_props[relationship_type]['cm_std'],
        'description': relationship_props[relationship_type]['description'],
        'prior_probability': 1.0  # Default uniform prior
    }
    
    return hypothesis

# Calculate the likelihood of observed data under a specific hypothesis
def calculate_hypothesis_loglikelihood(hypothesis, observed_data):
    """
    Calculate the log-likelihood of observed data under a given relationship hypothesis.
    
    Args:
        hypothesis: Dictionary representing the relationship hypothesis
        observed_data: Dictionary with observed IBD data
        
    Returns:
        Log-likelihood score
    """
    # Extract relevant information
    observed_total_cm = observed_data['total_cm']
    observed_segments = observed_data['num_segments']
    
    # Use a normal distribution for total cM (simplified model)
    ll_total_cm = stats.norm.logpdf(
        observed_total_cm, 
        loc=hypothesis['expected_ibd'], 
        scale=hypothesis['std_dev']
    )
    
    # Use a Poisson distribution for segment count (simplified model)
    ll_segments = stats.poisson.logpmf(
        observed_segments,
        mu=hypothesis['expected_segments']
    )
    
    # Combine the log-likelihoods (assuming independence)
    # In practice, these are correlated, but this simplification works for demonstration
    total_ll = ll_total_cm + ll_segments
    
    return total_ll

# Compute the Bayes factor for comparing two hypotheses
def compute_bayes_factor(h1_ll, h2_ll):
    """
    Compute the Bayes factor for comparing two hypotheses.
    
    Args:
        h1_ll: Log-likelihood of hypothesis 1
        h2_ll: Log-likelihood of hypothesis 2
        
    Returns:
        Bayes factor (in linear space)
    """
    # Bayes factor = P(data|H1) / P(data|H2)
    # In log space: BF = exp(log(P(data|H1)) - log(P(data|H2)))
    log_bf = h1_ll - h2_ll
    return math.exp(log_bf)

# Calculate posterior probabilities for multiple hypotheses
def calculate_posterior_probabilities(hypotheses, observed_data):
    """
    Calculate posterior probabilities for multiple hypotheses.
    
    Args:
        hypotheses: List of hypothesis dictionaries
        observed_data: Dictionary with observed IBD data
        
    Returns:
        List of hypotheses with posterior probabilities added
    """
    # Calculate log-likelihood for each hypothesis
    for h in hypotheses:
        h['log_likelihood'] = calculate_hypothesis_loglikelihood(h, observed_data)
    
    # Calculate log posterior (log likelihood + log prior)
    for h in hypotheses:
        h['log_posterior'] = h['log_likelihood'] + math.log(h['prior_probability'])
    
    # Normalize the posteriors
    log_posteriors = [h['log_posterior'] for h in hypotheses]
    log_norm_factor = logsumexp(log_posteriors)
    
    for h in hypotheses:
        h['posterior_probability'] = math.exp(h['log_posterior'] - log_norm_factor)
    
    # Sort by posterior probability (descending)
    hypotheses.sort(key=lambda x: x['posterior_probability'], reverse=True)
    
    return hypotheses

# Create several relationship hypotheses to test
def test_multiple_hypotheses(observed_data, relationship_types=None, prior_probs=None):
    """
    Test multiple relationship hypotheses and calculate posterior probabilities.
    
    Args:
        observed_data: Dictionary with observed IBD data
        relationship_types: List of relationship types to test
        prior_probs: Dictionary mapping relationship types to prior probabilities
        
    Returns:
        DataFrame with hypothesis testing results
    """
    if relationship_types is None:
        # Default to testing all common relationships
        relationship_types = [
            'parent-child', 'full-siblings', 'half-siblings', 
            '1st-cousins', '2nd-cousins', '3rd-cousins', 'unrelated'
        ]
    
    # Create hypothesis objects
    hypotheses = []
    for rel_type in relationship_types:
        hypothesis = create_relationship_hypothesis(rel_type)
        
        # Apply custom prior if provided
        if prior_probs and rel_type in prior_probs:
            hypothesis['prior_probability'] = prior_probs[rel_type]
            
        hypotheses.append(hypothesis)
    
    # Ensure priors sum to 1
    prior_sum = sum(h['prior_probability'] for h in hypotheses)
    for h in hypotheses:
        h['prior_probability'] /= prior_sum
    
    # Calculate posterior probabilities
    results = calculate_posterior_probabilities(hypotheses, observed_data)
    
    # Create a DataFrame for display
    result_df = pd.DataFrame([
        {
            'Relationship': h['name'],
            'Degree': h['degree'],
            'Prior': h['prior_probability'],
            'Log-Likelihood': h['log_likelihood'],
            'Posterior': h['posterior_probability']
        }
        for h in results
    ])
    
    return result_df

# Visualize hypothesis testing results
def visualize_hypothesis_results(result_df, title=None):
    """Visualize hypothesis testing results"""
    plt.figure(figsize=(12, 6))
    
    # Create a bar chart of posterior probabilities
    plt.bar(result_df['Relationship'], result_df['Posterior'], color='skyblue')
    
    plt.xlabel('Relationship Hypothesis')
    plt.ylabel('Posterior Probability')
    plt.title(title or 'Posterior Probabilities for Relationship Hypotheses')
    plt.xticks(rotation=45)
    plt.grid(axis='y', alpha=0.3)
    plt.tight_layout()
    
    # Add value labels on bars
    for i, v in enumerate(result_df['Posterior']):
        plt.text(i, v + 0.01, f"{v:.3f}", ha='center')
    
    plt.show()

# Demonstrate hypothesis testing with our example data
print("Testing multiple relationship hypotheses on example data:")
print(f"Total cM: {example_data['total_cm']:.1f}")
print(f"Number of segments: {example_data['num_segments']}")

# Test with uniform priors
result_uniform = test_multiple_hypotheses(example_data)
print("\nResults with uniform priors:")
print(result_uniform)

# Visualize results
visualize_hypothesis_results(result_uniform, title="Posterior Probabilities (Uniform Priors)")

# Now test with informative priors based on other information
# Example: We have documentary evidence suggesting half-siblings or 1st cousins
informative_priors = {
    'parent-child': 0.05,
    'full-siblings': 0.1,
    'half-siblings': 0.4,  # Higher prior for half-siblings
    '1st-cousins': 0.3,    # Higher prior for 1st cousins
    '2nd-cousins': 0.1,
    '3rd-cousins': 0.03,
    'unrelated': 0.02
}

result_informative = test_multiple_hypotheses(example_data, prior_probs=informative_priors)
print("\nResults with informative priors:")
print(result_informative)

# Visualize results with informative priors
visualize_hypothesis_results(result_informative, title="Posterior Probabilities (Informative Priors)")

# Compare the effect of priors
print("\nEffect of Priors on Hypothesis Testing:")
print("Uniform Priors - Top relationship: " + 
      f"{result_uniform.iloc[0]['Relationship']} (P={result_uniform.iloc[0]['Posterior']:.3f})")
print("Informative Priors - Top relationship: " + 
      f"{result_informative.iloc[0]['Relationship']} (P={result_informative.iloc[0]['Posterior']:.3f})")

# Calculate Bayes factor for top two hypotheses (with uniform priors)
if len(result_uniform) >= 2:
    top_rel = result_uniform.iloc[0]['Relationship']
    second_rel = result_uniform.iloc[1]['Relationship']
    
    top_ll = result_uniform.iloc[0]['Log-Likelihood']
    second_ll = result_uniform.iloc[1]['Log-Likelihood']
    
    bf = compute_bayes_factor(top_ll, second_ll)
    
    print(f"\nBayes Factor ({top_rel} vs. {second_rel}): {bf:.2f}")
    
    # Interpret Bayes factor
    if bf > 100:
        interpretation = "Very strong evidence for " + top_rel
    elif bf > 10:
        interpretation = "Strong evidence for " + top_rel
    elif bf > 3:
        interpretation = "Moderate evidence for " + top_rel
    else:
        interpretation = "Weak or inconclusive evidence"
        
    print(f"Interpretation: {interpretation}")

## Part 5: Visualizing Uncertainty in Relationship Predictions

Effectively communicating uncertainty in relationship predictions is crucial for users to make informed decisions. Bonsai v3 implements various visualization techniques to represent confidence levels and uncertainty in its predictions:

1. **Probability Distributions**: Visualizing the full distribution of relationship probabilities
2. **Confidence Regions**: Highlighting confidence intervals in visualizations
3. **Color Coding**: Using color to encode confidence levels in pedigree visualizations
4. **Alternative Hypotheses**: Showing multiple plausible pedigree structures

Let's explore how Bonsai helps users interpret results by visually representing uncertainty:

In [ ]:
# Visualizing Uncertainty in Relationship Predictions
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.patches as mpatches
import networkx as nx
import random
from matplotlib.colors import LinearSegmentedColormap

# Create a dataset of multiple samples with various confidence levels
np.random.seed(42)
random.seed(42)

# Function to generate a dataset with varying confidence levels
def generate_test_dataset(n_samples=20):
    """
    Generate a dataset of IBD comparisons with varying confidence levels.
    
    Args:
        n_samples: Number of sample pairs to generate
        
    Returns:
        DataFrame with relationship predictions and confidence metrics
    """
    # List of possible true relationships
    relationships = ['parent-child', 'full-siblings', 'half-siblings', 
                    '1st-cousins', '2nd-cousins', '3rd-cousins', 'unrelated']
    
    # List to store results
    results = []
    
    for i in range(n_samples):
        # Select a random true relationship
        true_rel = random.choice(relationships)
        
        # Generate a sample for this relationship
        sample_data = simulate_ibd_data(true_rel)
        
        # Calculate likelihoods
        ll_scores = calculate_likelihood_scores(sample_data)
        
        # Calculate normalized probabilities
        probs = normalize_log_likelihoods(ll_scores)
        
        # Find predicted relationship (highest probability)
        pred_rel = max(probs.items(), key=lambda x: x[1])[0]
        pred_prob = probs[pred_rel]
        
        # Calculate confidence ratio (top / second)
        sorted_probs = sorted(probs.items(), key=lambda x: x[1], reverse=True)
        if len(sorted_probs) >= 2 and sorted_probs[1][1] > 0:
            conf_ratio = sorted_probs[0][1] / sorted_probs[1][1]
        else:
            conf_ratio = float('inf')
        
        # Calculate entropy (measure of uncertainty)
        entropy = -sum(p * np.log2(p) for p in probs.values() if p > 0)
        
        # Store result
        results.append({
            'id': f"Sample-{i+1}",
            'true_relationship': true_rel,
            'predicted_relationship': pred_rel,
            'prediction_probability': pred_prob,
            'confidence_ratio': conf_ratio,
            'entropy': entropy,
            'total_cm': sample_data['total_cm'],
            'num_segments': sample_data['num_segments'],
            'correct': true_rel == pred_rel
        })
    
    # Convert to DataFrame
    df = pd.DataFrame(results)
    
    # Add a confidence category for easier visualization
    df['confidence_category'] = pd.cut(
        df['confidence_ratio'], 
        bins=[0, 2, 10, 100, float('inf')],
        labels=['Low', 'Moderate', 'High', 'Very High']
    )
    
    return df

# Generate test dataset
test_data = generate_test_dataset(n_samples=30)

# Display summary of test data
print("Generated test dataset with varying confidence levels:")
print(test_data[['id', 'true_relationship', 'predicted_relationship', 
                'confidence_ratio', 'confidence_category', 'correct']].head(10))

# Calculate overall accuracy
accuracy = test_data['correct'].mean()
print(f"\nOverall prediction accuracy: {accuracy:.2%}")

# Calculate accuracy by confidence category
accuracy_by_conf = test_data.groupby('confidence_category')['correct'].mean()
print("\nAccuracy by confidence category:")
print(accuracy_by_conf)

# 1. Visualize relationship prediction probability vs confidence ratio
plt.figure(figsize=(10, 6))
plt.scatter(
    test_data['prediction_probability'], 
    test_data['confidence_ratio'],
    c=test_data['correct'].map({True: 'green', False: 'red'}),
    alpha=0.7,
    s=80
)

plt.xscale('linear')
plt.yscale('log')
plt.xlabel('Prediction Probability')
plt.ylabel('Confidence Ratio (log scale)')
plt.title('Relationship Prediction Confidence')
plt.grid(True, alpha=0.3)

# Add legend
green_patch = mpatches.Patch(color='green', label='Correct Prediction')
red_patch = mpatches.Patch(color='red', label='Incorrect Prediction')
plt.legend(handles=[green_patch, red_patch])

# Add confidence region thresholds
plt.axhline(y=2, color='gray', linestyle='--', alpha=0.5)
plt.axhline(y=10, color='gray', linestyle='--', alpha=0.5)
plt.axhline(y=100, color='gray', linestyle='--', alpha=0.5)

plt.text(0.05, 1.5, 'Low Confidence', ha='left', va='top', alpha=0.7)
plt.text(0.05, 5, 'Moderate Confidence', ha='left', va='top', alpha=0.7)
plt.text(0.05, 50, 'High Confidence', ha='left', va='top', alpha=0.7)
plt.text(0.05, 500, 'Very High Confidence', ha='left', va='top', alpha=0.7)

plt.tight_layout()
plt.show()

# 2. Visualize prediction accuracy by confidence category
plt.figure(figsize=(10, 6))
sns.barplot(
    x=accuracy_by_conf.index,
    y=accuracy_by_conf.values,
    palette=['#ff9999', '#ffcc99', '#99cc99', '#66bb66']
)

plt.xlabel('Confidence Category')
plt.ylabel('Prediction Accuracy')
plt.title('Accuracy by Confidence Level')
plt.ylim(0, 1.05)

# Add value labels on bars
for i, v in enumerate(accuracy_by_conf):
    plt.text(i, v + 0.02, f"{v:.2%}", ha='center')

plt.tight_layout()
plt.show()

# 3. Visualize a heat map of confidence by true and predicted relationship
# Create a pivot table of average confidence ratio by true and predicted relationship
conf_heatmap = test_data.pivot_table(
    index='true_relationship',
    columns='predicted_relationship',
    values='confidence_ratio',
    aggfunc='mean'
)

# Plot heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(
    conf_heatmap,
    annot=True,
    fmt=".1f",
    cmap="YlGnBu",
    linewidths=.5,
    cbar_kws={'label': 'Avg. Confidence Ratio'}
)

plt.title('Confidence by True vs. Predicted Relationship')
plt.tight_layout()
plt.show()

# 4. Visualize relationship probabilities for a specific example
# Let's select a sample with moderate confidence
moderate_conf_sample = test_data[test_data['confidence_category'] == 'Moderate'].iloc[0]
sample_id = moderate_conf_sample['id']

# Get the sample data
sample_idx = int(sample_id.split('-')[1]) - 1
sample_rel = moderate_conf_sample['true_relationship']
sample_data = simulate_ibd_data(sample_rel, num_segments=test_data.loc[sample_idx, 'num_segments'])

# Calculate the probabilities
sample_ll = calculate_likelihood_scores(sample_data)
sample_probs = normalize_log_likelihoods(sample_ll)

# Plot the probability distribution
plt.figure(figsize=(12, 6))

# Sort relationships by probability
sorted_rels = sorted(sample_probs.items(), key=lambda x: x[1], reverse=True)
rels = [r for r, _ in sorted_rels]
probs = [p for _, p in sorted_rels]

# Create colormap based on confidence
colors = []
for i, (rel, prob) in enumerate(sorted_rels):
    if i == 0:
        colors.append('#3366cc')  # Blue for highest probability
    else:
        # Calculate confidence ratio with top relationship
        ratio = sorted_rels[0][1] / prob if prob > 0 else float('inf')
        if ratio > 100:
            colors.append('#cccccc')  # Light gray for very low probability
        elif ratio > 10:
            colors.append('#999999')  # Medium gray
        else:
            colors.append('#666666')  # Dark gray for more plausible alternatives

plt.bar(rels, probs, color=colors)
plt.xlabel('Relationship Type')
plt.ylabel('Probability')
plt.title(f'Relationship Probability Distribution (Sample {sample_id})')
plt.xticks(rotation=45)
plt.grid(axis='y', alpha=0.3)

# Add value labels on bars
for i, v in enumerate(probs):
    plt.text(i, v + 0.01, f"{v:.3f}", ha='center')

# Highlight true relationship
true_rel_idx = rels.index(moderate_conf_sample['true_relationship']) if moderate_conf_sample['true_relationship'] in rels else -1
if true_rel_idx >= 0:
    plt.gca().get_xticklabels()[true_rel_idx].set_color('red')
    plt.gca().get_xticklabels()[true_rel_idx].set_weight('bold')

plt.tight_layout()
plt.show()

# 5. Simulate a simple pedigree visualization with confidence color coding
def visualize_pedigree_with_confidence(relationships_with_confidence):
    """
    Create a simplified pedigree visualization with confidence color coding.
    
    Args:
        relationships_with_confidence: List of (id1, id2, relationship, confidence) tuples
    """
    # Create network graph
    G = nx.Graph()
    
    # Add all unique individuals
    all_ids = set()
    for id1, id2, _, _ in relationships_with_confidence:
        all_ids.add(id1)
        all_ids.add(id2)
    
    for person_id in all_ids:
        G.add_node(person_id)
    
    # Create colormap for confidence levels
    confidence_cmap = LinearSegmentedColormap.from_list(
        'confidence', 
        [(0, '#ff9999'), (0.33, '#ffcc99'), (0.67, '#99cc99'), (1, '#339933')]
    )
    
    # Add edges with color based on confidence
    edge_colors = []
    edge_widths = []
    edge_labels = {}
    
    for id1, id2, rel, conf in relationships_with_confidence:
        G.add_edge(id1, id2)
        
        # Normalize confidence for color mapping (0-100 scale)
        norm_conf = min(1.0, conf / 100)
        edge_colors.append(confidence_cmap(norm_conf))
        
        # Edge width based on relationship closeness
        if rel == 'parent-child' or rel == 'full-siblings':
            width = 3.0
        elif rel == 'half-siblings' or rel == '1st-cousins':
            width = 2.0
        else:
            width = 1.0
        
        edge_widths.append(width)
        
        # Edge label with relationship and confidence
        conf_str = f"{conf:.1f}"
        if conf > 100:
            conf_str = ">100"
        edge_labels[(id1, id2)] = f"{rel}\n({conf_str}x)"
    
    # Create plot
    plt.figure(figsize=(12, 10))
    pos = nx.spring_layout(G, seed=42)  # Use spring layout
    
    # Draw nodes
    nx.draw_networkx_nodes(G, pos, node_size=500, node_color='lightblue')
    
    # Draw edges with custom colors and widths
    for i, (u, v) in enumerate(G.edges()):
        nx.draw_networkx_edges(
            G, pos, edgelist=[(u, v)], 
            width=edge_widths[i], 
            edge_color=[edge_colors[i]]
        )
    
    # Draw node labels
    nx.draw_networkx_labels(G, pos, font_size=10)
    
    # Draw edge labels
    nx.draw_networkx_edge_labels(
        G, pos, 
        edge_labels=edge_labels,
        font_size=8
    )
    
    plt.title("Pedigree with Confidence Visualization")
    plt.axis('off')
    plt.tight_layout()
    plt.show()

# Create a sample pedigree with confidence values
example_pedigree = [
    ('A', 'B', 'parent-child', 150.0),    # Very high confidence
    ('A', 'C', 'parent-child', 120.0),    # Very high confidence
    ('B', 'D', 'parent-child', 80.0),     # High confidence
    ('C', 'E', 'parent-child', 15.0),     # Moderate confidence
    ('D', 'F', 'parent-child', 4.0),      # Low confidence
    ('E', 'F', '1st-cousins', 2.5),       # Low confidence
    ('G', 'F', 'half-siblings', 6.0)      # Moderate confidence
]

# Visualize pedigree with confidence color coding
visualize_pedigree_with_confidence(example_pedigree)

print("\nInterpreting Visualization Confidence:")
print("- Green connections: High confidence relationships")
print("- Yellow connections: Moderate confidence relationships")
print("- Red connections: Low confidence relationships")
print("- Edge thickness indicates relationship closeness")
print("- Labels show relationship type and confidence ratio")

print("\nBest Practices for Interpreting Results:")
print("1. Focus on high-confidence predictions first (green connections)")
print("2. Use age information to disambiguate uncertain relationships")
print("3. Consider alternative hypotheses for low-confidence predictions")
print("4. Look for patterns of errors (e.g., consistent under-estimation of degree)")
print("5. Integrate documentary evidence to refine genetic predictions")
print("6. Recognize that small segments (<7 cM) may increase false positives")
print("7. Consider endogamy and population structure as potential confounders")

## Conclusion: Making Sense of Genetic Relationship Predictions

In this lab, we've explored how Bonsai v3 interprets genetic data to infer relationships between individuals and assigns confidence to these predictions. We've covered:

1. **Likelihood-Based Inference**: We learned how Bonsai calculates likelihoods for different relationship hypotheses and uses these to determine the most probable relationship.

2. **Confidence Intervals**: We explored methods for calculating confidence bounds on relationship degree estimates, providing a range of plausible relationships rather than a single point estimate.

3. **Age-Based Constraints**: We examined how age information can significantly enhance the accuracy of relationship predictions by ruling out implausible relationships.

4. **Multiple Hypothesis Testing**: We demonstrated systematic approaches for comparing alternative relationship hypotheses using Bayes factors and posterior probabilities.

5. **Visualizing Uncertainty**: We explored various techniques for visually representing confidence levels in relationship predictions, helping users understand the reliability of results.

Effective interpretation of genetic relationship data requires understanding both the statistical models and their limitations. By properly interpreting confidence measures and incorporating multiple sources of evidence, users can make more informed decisions about family connections based on genetic data.

Remember these key principles:

- Relationship predictions are probabilistic, not deterministic
- Higher confidence (larger likelihood ratios) indicates more reliable predictions
- Multiple sources of evidence (genetic data, age information, documentary evidence) should be integrated
- Visual representations help communicate uncertainty effectively
- Always consider alternative explanations for ambiguous predictions

As computational methods for genetic genealogy continue to advance, the ability to accurately assess and communicate confidence in relationship predictions remains crucial for responsible interpretation of results.

In [ ]:
# Try to use Bonsai's PwLogLike class for relationship inference
try:
    from utils.bonsaitree.bonsaitree.v3.likelihoods import PwLogLike
    from utils.bonsaitree.bonsaitree.v3.ibd import IBDSegment

    print("✅ Successfully imported PwLogLike and IBDSegment classes")
    
    # Convert our simulated IBD data to Bonsai's IBDSegment format
    def convert_to_bonsai_segments(ibd_data):
        """Convert simulated IBD data to Bonsai IBDSegment objects"""
        bonsai_segments = []
        
        for segment in ibd_data['segments']:
            try:
                ibd_seg = IBDSegment(
                    chrom=segment['chromosome'],
                    start_pos=segment['start_pos'],
                    end_pos=segment['end_pos'],
                    cm_length=segment['cm'],
                    snp_count=segment['snps']
                )
                bonsai_segments.append(ibd_seg)
            except Exception as e:
                print(f"Error creating IBDSegment: {e}")
                
        return bonsai_segments
    
    # Using the example data we created earlier
    bonsai_segments = convert_to_bonsai_segments(example_data)
    print(f"Created {len(bonsai_segments)} Bonsai IBDSegment objects")
    
    # Create a PwLogLike object to calculate likelihoods
    try:
        # This is a simplified call - actual parameters may vary based on the implementation
        pw_log_like = PwLogLike(bonsai_segments)
        
        # Get likelihood scores for different degrees
        rel_scores = {}
        for degree in range(1, 5):  # Typically 1st to 4th degree
            score = pw_log_like.get_loglike(degree)
            rel_scores[f"degree-{degree}"] = score
            
        print("\nBonsai PwLogLike Scores:")
        for rel, score in sorted(rel_scores.items(), key=lambda x: x[1], reverse=True):
            print(f"{rel:<10}: {score:.2f}")
        
        # Get the most likely degree
        most_likely_degree = max(rel_scores.items(), key=lambda x: x[1])[0]
        print(f"\nMost likely relationship degree: {most_likely_degree}")
        
        # Calculate confidence interval if available
        if hasattr(pw_log_like, 'get_degree_conf_int'):
            conf_int = pw_log_like.get_degree_conf_int(0.95)  # 95% confidence interval
            print(f"95% Confidence Interval: {conf_int}")
    
    except Exception as e:
        print(f"Error using PwLogLike: {e}")
        print("The PwLogLike interface may have changed. Check the class documentation.")
    
except ImportError as e:
    print(f"❌ Failed to import Bonsai classes: {e}")
    print("Falling back to simplified implementation...")
    
    # Fallback to our simplified implementation
    print("\nUsing simplified likelihood calculation:")
    
    # Calculate simplified likelihood
    simple_ll = calculate_likelihood_scores(example_data)
    simple_normalized = normalize_log_likelihoods(simple_ll)
    
    # Display results
    for rel, prob in sorted(simple_normalized.items(), key=lambda x: x[1], reverse=True)[:3]:
        print(f"{rel:<15}: {prob:.4f} ({prob*100:.1f}%)")
    
    # Plot simplified relationship probabilities
    plt.figure(figsize=(10, 6))
    
    # Get top 5 relationships
    top_rels = sorted(simple_normalized.items(), key=lambda x: x[1], reverse=True)[:5]
    rels = [r for r, _ in top_rels]
    probs = [p for _, p in top_rels]
    
    # Create bar chart
    plt.bar(rels, probs, color='lightgreen')
    plt.xlabel('Relationship Type')
    plt.ylabel('Probability')
    plt.title('Top 5 Relationship Probabilities (Simplified Model)')
    plt.xticks(rotation=45)
    plt.grid(axis='y', alpha=0.3)
    plt.tight_layout()
    plt.show()